# Bond Pricing

**The bond price is the discounted value of its cash flows.**

1. Obtain the cash flows;
2. Obtain the discount rates;
3. Obtain the Present Value of the cash flows.

## 1. The Cash Flows
- Maturity date
- Coupon rate
- Principal (face value)

In [5]:
def bond_cash_flows(maturity, principal, coupon_rate, coupons_per_year=2):
    """Returns a series of cash flows generated by a bond."""
    import numpy as np
    import pandas as pd
    n_coupons = round(maturity*coupons_per_year)
    coupon_pmt = principal*(coupon_rate/coupons_per_year)
    coupon_index = np.arange(1, n_coupons + 1)
    cash_flows = pd.Series(data = coupon_pmt, index = coupon_index)
    cash_flows.iloc[-1] += principal
    return pd.Series(cash_flows)

**Example 2.4:**
Consider a bond with 5% annual coupon rate, 10-y maturity and $1000 face value.

In [23]:
bond_cash_flows(maturity=10, principal=1000, coupon_rate=0.05, coupons_per_year=1)

1       50.0
2       50.0
3       50.0
4       50.0
5       50.0
6       50.0
7       50.0
8       50.0
9       50.0
10    1050.0
dtype: float64

## 2. Discounting cash flows with identical discount rate and cash flows

\begin{equation*}
P_0=C\times\frac{1}{y}\times\Bigl(1-\frac{1}{(1+y)^T}\Bigr)+\frac{N}{(1+y)^T}\\  
\text{where }P_0\text{ is the PV of the bond, }T\text{ is the maturity, }N\text{ is the nominal value, }C=c\times N\text{ is the coupon payment, }c\text{ is the coupon rate and }y\text{ is the discount rate}
\end{equation*}

In [2]:
def bond_price_math(maturity, principal, coupon_rate, coupons_per_year, discount_rate):
    """Price a bond with identical discount rate and cash flows over periods."""
    import pandas as pd
    coupon_pmt = principal*(coupon_rate/coupons_per_year)
    discount_parameter = (1/(1+discount_rate)**maturity)
    coupons_discounted_cf = coupon_pmt * (1/discount_rate) * (1 - discount_parameter)
    principal_discounted_cf = principal * discount_parameter
    return coupons_discounted_cf + principal_discounted_cf 

**Example 2.4:**
Consider a bond with 5% annual coupon rate, 10-y maturity, $1000 face value and discount rate equal 6%.

In [4]:
bond_price_math(maturity=10, principal=1000, coupon_rate=0.05, coupons_per_year=1, discount_rate = 0.06)

926.399129485853

## 3. Pricing bonds by discounted cash flows

In [3]:
def discount(t, r):
    """Discounted rate."""
    import pandas as pd
    discounts = pd.Series([(r+1)**-i for i in t])
    discounts.index = t
    return discounts

def pv(flows, r):
    """Compute the pv of cash flows indexed by time."""
    dates = flows.index
    discounts = discount(dates, r)
    return discounts.multiply(flows, axis='rows').sum()

def bond_price(maturity, principal, coupon_rate, coupons_per_year, discount_rate):
    """Price a bond based on its cash flows."""
    import pandas as pd
    if maturity <= 0:
        return principal*(1 + coupon_rate/coupons_per_year)
    else:
        cash_flows = bond_cash_flows(maturity, principal,
                                     coupon_rate, coupons_per_year)
        return pv(cash_flows, discount_rate/coupons_per_year)
 

In [6]:
bond_price(maturity=10, principal=1, coupon_rate=0.02, coupons_per_year=1, discount_rate = 0.025)

0.9562396803451462

### Nominal vs Real interest rates

\begin{equation}
1+R_r=\frac{1+R_n}{1+i}
\end{equation}

In [29]:
def real_interest_rate(nominal_rate, inflation_rate):
    """Compute real interest rate over periods."""
    import pandas as pd
    return (1 + nominal_rate)/(1 + inflation_rate) - 1

In [35]:
import pandas as pd
nominal_rates = pd.Series(data = [.05, .05, .045, .055])
inflation_rates = pd.Series(data = [.02, .018, 0.024, 0.015])
real_interest_rate(nominal_rates, inflation_rates)

0    0.029412
1    0.031434
2    0.020508
3    0.039409
dtype: float64

## Taxonomy of Rates

**Current Yield**:
\begin{equation}
Y_c=\frac{cN}{P}\\
\text{where }c\text{ is the coupon rate, }N\text{ is the nominal value and }P\text{ is the current price}
\end{equation}
**Yield to Maturity (YTM)**:
Is the IRR of the series of cash flows. It's expressed on a yearly basis.


In [58]:
def bond_ytm(price, principal, maturity, coupon_rate, coupons_per_year, discount_rate_guess=0.05):
    """Compute the YTM using the Newton-Raphson method"""
    import scipy.optimize as optimize
    cash_flows = bond_cash_flows(maturity, principal, coupon_rate, coupons_per_year)
    return optimize.newton(lambda ytm: pv(cash_flows, ytm) - price, discount_rate_guess)* coupons_per_year

**Example 2.16** Consider a bond with 8% semiannual coupon rate, 2-y maturity, $1000 face value and price 103-23.

In [104]:
price = (103 + 23/32)/100 * 1000
bond_ytm(price, principal=1000, maturity=2, coupon_rate=0.08, coupons_per_year=2, discount_rate_guess=0.04)

0.03996539411117196

**Example 2.17**  consider a $1,000 face value 3-year bond with 10% annual coupon, which sells for 101.

In [100]:
price = 101/100 * 1000
bond_ytm(price, principal=1000, maturity=3, coupon_rate=0.10, coupons_per_year=1, discount_rate_guess=0.07)

0.09600708826627964

## Macauly Duration

Given a bond, the macauly duration of that bond is the weighted sum of its cash flows by time divided by its present value.

\begin{equation}
Mac.duration=\frac{\sum_i^T{(t_i\times VP_i)}}{VP}
\end{equation}

## Modified Duration

Mesures the bond's sensitivity to changes in maturity or coupon/interest rates.

\begin{equation}
Mod.duration=\frac{Mac.duration}{1+\frac{YTM}{T}}
\end{equation}

In [72]:
def macauly_duration(cash_flows, discount_rate):
    """Compute the Macauly duration of a bond"""
    import numpy as np
    discounted_flows = discount(cash_flows.index, discount_rate)*cash_flows
    weights = discounted_flows/discounted_flows.sum()
    return np.average(pd.Series(cash_flows.index), weights=weights)

In [78]:
macauly_duration(bond_cash_flows(maturity=10, principal=1000, coupon_rate=0.05, coupons_per_year=1), 0.04)

8.190898824083233

In [80]:
def modified_duration(macauly_duration, YTM, maturity):
    """Compute the modified duration of a bond"""
    return macauly_duration/(1 + YTM/maturity)

## Effective duration

\begin{equation}
MD=-\frac{\Delta P}{P\times \Delta T} \\
\text{where }\Delta T\text{ is the interest variation}
\end{equation}

* BPV (or PV01): price change in % or $ for 0.01% (one basis point)

\begin{equation}
PV01=S\times P\times 0.0001
\end{equation}

**Example:**
A bond is issued with a coupon of X% and maturity of 10y. The bond sensitivity is 8. At a yield of 4% the price is 98%. Estimate the coupon.

\begin{equation}
MD\times(c-YTM)\times(Price_0)=-(Price_0-Price_1)\Rightarrow c=\frac{-(Price_0-Price_1)-(MD\times YTM)\times(Price_0)}{MD\times(Price_0)}
\end{equation}

In [105]:
(1-.98-8*0.04*1)/(8*1)

-0.0375

**Example**: A bond is quoted with a price of 110%. Its modified duration (sensitivy) is 5. If the yield of the bond goes down by 10 bp, what will be the price?
\begin{equation}
1.1-P=-5\times0.001\times1.1\Rightarrow P=5\times0.001\times1.1+1.1
\end{equation}

In [114]:
5*0.001*1.1+1.1

1.1055000000000001

**Example**
- coupon: 0.02
- Nominal Value: 1000
- maturity: 10y
- YTM = 0.01

In [116]:
bond_price(maturity=10, principal=1000, coupon_rate=0.02, coupons_per_year=1, discount_rate = 0.01)

1094.7130453070167

In [118]:
bond_price_math(maturity=10, principal=1000, coupon_rate=0.02, coupons_per_year=1, discount_rate = 0.01)

1094.7130453070167